In [8]:
from telegram.ext import Updater,CallbackContext,CommandHandler
from telegram import Update
import logging
import time
import hmac
import requests
import pandas as pd
from datetime import datetime
import urllib.parse
import asyncio
import dataframe_image as dfi

In [9]:
api_key = "zi6mtjXRTcRrS6O_TMrRrK-PiChM7d2msxmBlTun"
api_secret = "4eI4QUG7eOumG_tq_z9TsjM93TALdhEGM0qEPGfj"

In [10]:
class FTX_Client_Hicham:
    api_endpoint_base = "https://ftx.com/api/"

    def __init__(self,api_key:str,api_secret:str,subaccount:str=None):
        """Indiquer les clés API et éventuellement un subaccount"""
        self._session = requests.Session()
        self._api_key = api_key
        self._api_secret = api_secret
        self._subaccount = subaccount

    def get_open_orders(self):
        ''' Cette fonction permet de récupérer les ordres qui sont actuellement ouverts'''
        url = self.api_endpoint_base + "orders"
        request = requests.Request('GET', url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        if len(df_data)!=0:
            df_data.drop(['clientId','reduceOnly','liquidation','postOnly','ioc',"future","createdAt","avgFillPrice","remainingSize","filledSize"],axis=1,inplace=True) 
            df_data.set_index("id",inplace=True)    
        return df_data

    def get_positions(self):
        """Cette fonction permet de récupérer les positions actuellement ouvertes sur des contrats futures"""
        url = self.api_endpoint_base + "positions"
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        return df_data[df_data["size"]>0]

    def cancel_order(self,order_id):
        """ Cette fonction permet d'annuler un ordre s'il n'a pas déjà été exécuté, l'ID de l'ordre est obtenu dans le résultat de la fonction send_order"""
        url = self.api_endpoint_base + "orders/" + str(order_id)
        request = requests.Request('DELETE',url)
        data = self.send_request(request)
        #df_data = pd.DataFrame(data)
        return data


    def get_balances(self):
        """ Cette fonction permet de récupérer le solde du compte"""
        url = self.api_endpoint_base + "wallet/balances"
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        return df_data[df_data["total"]>0]

    def get_order_history(self):
        """ Cette fonction permet de récupérer l'historique des ordres passés (sans forcément avoir été exécutés)"""
        url = self.api_endpoint_base + "orders/history"
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)     
        return df_data   

    def get_last_n_funding_rates(self,market,n):
        ''' Cette fonction permet de récupérer les n derniers funding rates sur un future perp'''
        url = self.api_endpoint_base + "funding_rates?future=" + market
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        df_data.set_index("time",inplace=True)
        return df_data[:n]['rate'].to_numpy()

    def send_order(self,market:str,side:str,price:str,type:str,size:str):
        """ Cette fonction permet d'envoyer un ordre:
        market : paire (BTC/USD) ou future (BTC-PERP)      
        side: buy ou sell
        price: prix (mettre 0 pour un ordre market)
        type: limit ou market
        size: quantité
        return: renvoie un récapitulatif de l'ordre passé contenant notamment son ID 
        """
        url = self.api_endpoint_base + "orders"
        params = {"market":market,"side":side,"price":price,"type":type,"size":size,"postOnly":True}
        request = requests.Request("POST",url,json=params)
        data = self.send_request(request)
        return data

    def process_response(self,response):
        ''' Cette fonction sert à traiter la réponse du serveur FTX'''
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']

    def send_request(self,request):
        """ Cette fonction sert à envoyer les requêtes au serveur FTX"""
        ts = int(time.time() * 1000)
        prepared = request.prepare()

        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()

        if prepared.body:
            signature_payload += prepared.body

        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()

        prepared.headers[f'FTX-KEY'] = self._api_key
        prepared.headers[f'FTX-SIGN'] = signature
        prepared.headers[f'FTX-TS'] = str(ts)
        if self._subaccount :
            prepared.headers[f'FTX-SUBACCOUNT'] = urllib.parse.quote(self._subaccount)

        response = self._session.send(prepared)
        return self.process_response(response) 
    
    def get_live_quote(self,market:str):
        """Permet d'avoir les quotes pour les spots et les futures, par ex: get_quote("BTC/USD") ou get_quote("BTC-PERP")"""
        url = self.api_endpoint_base + "markets/" + market
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame.from_dict(data,orient="index")
        return df_data

    def get_historical_data(self,market:str,resolution:int,number:int):
        """Permet d'avoir les données historiques sur une paire spot ou sur un future, la résolution est à indiquer en secondes"""
        url = self.api_endpoint_base + "markets/" + market + "/candles?resolution=" + str(resolution) + "&limit=" + str(number)
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        df_data['date'] = pd.to_datetime(df_data["time"]/1000,unit='s',origin='unix')
        df_data.drop(['startTime','time'],axis=1,inplace=True)  
        df_data.set_index('date',inplace=True)
        return df_data

    def get_last_n_minutes_future_spot_spread(self,market_fut:str,market_spot:str,n:int):
        """Permet d'avoir le spread entre un future et le spot associé pour les n dernières minutes"""
        fut = self.get_historical_data(market_fut,60,n)
        spot = self.get_historical_data(market_spot,60,n)
        return ((fut-spot)/spot)["close"]

    def get_futures(self):
        url = self.api_endpoint_base + "futures"
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        return df_data

    def get_top_n_futures_by_volume(self,n):
        return self.get_futures().sort_values("volumeUsd24h",ascending=False)[:n]

    def get_available_opportunities(self):
        top_50_futures = self.get_top_n_futures_by_volume(30)
        Coins = top_50_futures["underlying"]
        Opportunities = []
        for coin in Coins :
            try : 
                if (self.get_last_n_minutes_future_spot_spread(coin+"-PERP",coin+"/USD",5)>0).all() == True:
                    Opportunities.append(coin)
            except :
                pass
        return Opportunities

In [11]:

client = FTX_Client_Hicham(api_key,api_secret)

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',level=logging.INFO)

updater = Updater(token=my_token, use_context=True)
dispatcher = updater.dispatcher

def start(update: Update, context: CallbackContext):
    context.bot.send_message(chat_id=update.effective_chat.id, text=str(client.get_live_quote("BTC/USD")[0]["last"] )+ " $")

def balance(update: Update, context: CallbackContext):
    dfi.export(client.get_balances(),"balances.png")
    context.bot.send_photo(chat_id=update.effective_chat.id, photo = open("./balances.png",'rb'))

def hello(update: Update, context: CallbackContext):
    context.bot.send_message(chat_id = update.effective_chat.id, text = "Salut, t'es prêt pour trader ?")

def arb_opportunities(update: Update, context: CallbackContext):
    context.bot.send_message(chat_id = update.effective_chat.id, text = ', '.join(client.get_available_opportunities()))

start_handler = CommandHandler('prix_btc', start)
balance_handler = CommandHandler("balances",balance)
hello_handler = CommandHandler("hello",hello)
arb_opportunities_handler = CommandHandler("arb_opportunities",arb_opportunities)

dispatcher.add_handler(start_handler)
dispatcher.add_handler(balance_handler)
dispatcher.add_handler(hello_handler)
dispatcher.add_handler(arb_opportunities_handler)

updater.start_polling()


2022-06-02 10:32:25,983 - apscheduler.scheduler - INFO - Scheduler started
